In [1]:
import os
import scenicplus
import scanpy as sc
import warnings
import pandas as pd
import matplotlib as mpl
import pickle
import dill
import sys

warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(5, 5), facecolor='white')

work_dir = '/data/leuven/345/vsc34561/ibp-2022-data/'
rnaseq_dir = work_dir + 'sn_rna_seq/'
rnaseq_output = rnaseq_dir + 'processed/'
atacseq_dir = work_dir + 'sn_atac_seq/'
state_dir = work_dir + 'state/'
archive_dir = state_dir + 'archive/'
local_state_dir = '/data/leuven/338/vsc33838/mirror/state'
motif_path = '/data/leuven/338/vsc33838/mirror/state/motifs/'
scratch_dir = os.getenv('VSC_SCRATCH')

In [2]:
adata = sc.read_h5ad(os.path.join(state_dir, 'alzheimers.h5ad'))
cistopic_obj = dill.load(open(os.path.join(state_dir, 'cistopic_obj.pkl'), 'rb'))
menr = dill.load(open(os.path.join(motif_path, 'menr.pkl'), 'rb'))

In [3]:
from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
scplus_obj = create_SCENICPLUS_object(
    GEX_anndata = adata.raw.to_adata(),
    cisTopic_obj = cistopic_obj,
    menr = menr,
    multi_ome_mode = False,
    key_to_group_by = 'celltype',
    nr_cells_per_metacells = 5)

2022-11-11 11:14:43,925 cisTopic     INFO     Imputing drop-outs
2022-11-11 11:15:24,369 cisTopic     INFO     Scaling
2022-11-11 11:16:20,829 cisTopic     INFO     Keep non zero rows
2022-11-11 11:17:31,955 cisTopic     INFO     Imputed accessibility sparsity: 0.7658145431815311
2022-11-11 11:17:31,958 cisTopic     INFO     Create CistopicImputedFeatures object
2022-11-11 11:17:31,959 cisTopic     INFO     Making matrix sparse
2022-11-11 11:23:24,541 cisTopic     INFO     Done!
2022-11-11 11:23:24,771 create scenicplus object INFO     Following annotations were found in both assays under key celltype:
	INH, MG, EX, ODC, OPC, PER.END, ASC.
Keeping 60082 cells for RNA and 31142 for ATAC.
2022-11-11 11:24:46,983 create scenicplus object INFO     Automatically set `nr_metacells` to: ASC: 1860, EX: 594, INH: 532, MG: 1624, ODC: 6704, OPC: 358, PER.END: 132
2022-11-11 11:24:46,986 create scenicplus object INFO     Generating pseudo multi-ome data


In [12]:
ensembl_version_dict = {'105': 'http://www.ensembl.org',
                        '104': 'http://may2021.archive.ensembl.org/',
                        '103': 'http://feb2021.archive.ensembl.org/',
                        '102': 'http://nov2020.archive.ensembl.org/',
                        '101': 'http://aug2020.archive.ensembl.org/',
                        '100': 'http://apr2020.archive.ensembl.org/',
                        '99': 'http://jan2020.archive.ensembl.org/',
                        '98': 'http://sep2019.archive.ensembl.org/',
                        '97': 'http://jul2019.archive.ensembl.org/',
                        '96': 'http://apr2019.archive.ensembl.org/',
                        '95': 'http://jan2019.archive.ensembl.org/',
                        '94': 'http://oct2018.archive.ensembl.org/',
                        '93': 'http://jul2018.archive.ensembl.org/',
                        '92': 'http://apr2018.archive.ensembl.org/',
                        '91': 'http://dec2017.archive.ensembl.org/',
                        '90': 'http://aug2017.archive.ensembl.org/',
                        '89': 'http://may2017.archive.ensembl.org/',
                        '88': 'http://mar2017.archive.ensembl.org/',
                        '87': 'http://dec2016.archive.ensembl.org/',
                        '86': 'http://oct2016.archive.ensembl.org/',
                        '80': 'http://may2015.archive.ensembl.org/',
                        '77': 'http://oct2014.archive.ensembl.org/',
                        '75': 'http://feb2014.archive.ensembl.org/',
                        '54': 'http://may2009.archive.ensembl.org/'}

import pybiomart as pbm
def test_ensembl_host(scplus_obj, host, species):
    dataset = pbm.Dataset(name=species+'_gene_ensembl',  host=host)
    annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
    annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    annot['Chromosome'] = annot['Chromosome'].astype('str')
    filter = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
    annot = annot[~filter]
    annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    gene_names_release = set(annot['Gene'].tolist())
    ov=len([x for x in scplus_obj.gene_names if x in gene_names_release])
    print('Genes recovered: ' + str(ov) + ' out of ' + str(len(scplus_obj.gene_names)))
    return ov

n_overlap = {}
for version in ensembl_version_dict.keys():
    print(f'host: {version}')
    try:
        n_overlap[version] =  test_ensembl_host(scplus_obj, ensembl_version_dict[version], 'hsapiens')
    except:
        print('Host not reachable')
v = sorted(n_overlap.items(), key=lambda item: item[1], reverse=True)[0][0]
print(f"version: {v} has the largest overlap, use {ensembl_version_dict[v]} as biomart host")

host: 105
Genes recovered: 4561 out of 5354
host: 104
Genes recovered: 4590 out of 5354
host: 103
Genes recovered: 5188 out of 5354
host: 102


/data/leuven/338/vsc33838/miniconda3/envs/ibp-2022-env/lib/python3.9/site-packages/pybiomart/dataset.py:269: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  result = pd.read_csv(StringIO(response.text), sep='\t')


Genes recovered: 5196 out of 5354
host: 101
Genes recovered: 5208 out of 5354
host: 100
Host not reachable
host: 99
Genes recovered: 5230 out of 5354
host: 98
Genes recovered: 5265 out of 5354
host: 97
Genes recovered: 5274 out of 5354
host: 96
Genes recovered: 5305 out of 5354
host: 95
Genes recovered: 5336 out of 5354
host: 94
Genes recovered: 5352 out of 5354
host: 93
Genes recovered: 5295 out of 5354
host: 92
Genes recovered: 5285 out of 5354
host: 91
Genes recovered: 5242 out of 5354
host: 90
Genes recovered: 5221 out of 5354
host: 89
Host not reachable
host: 88
Host not reachable
host: 87
Host not reachable
host: 86
Host not reachable
host: 80
Genes recovered: 4502 out of 5354
host: 77
Genes recovered: 4426 out of 5354
host: 75
Host not reachable
host: 54
Host not reachable
version: 94 has the largest overlap, use http://oct2018.archive.ensembl.org/ as biomart host


In [4]:
biomart_host = "http://oct2018.archive.ensembl.org/"

In [ ]:
scplus_obj = dill.load(open(os.path.join(scratch_dir, 'scenicplus/scplus_obj_f.pkl'), 'rb'))
scplus_obj

In [ ]:
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    sys.stderr = open(os.path.join(scratch_dir, 'err.log'), "w")
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = os.path.join(local_state_dir, 'TF_names_v_1.01.txt'),
        save_path = os.path.join(scratch_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = os.path.join(local_state_dir, 'bin'),
        n_cpu = 30,
        _temp_dir = os.path.join(scratch_dir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    dill.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)
    raise(e)

2022-11-08 14:33:42,820 SCENIC+_wrapper INFO     /scratch/leuven/338/vsc33838/scenicplus folder already exists.
2022-11-08 14:33:42,821 SCENIC+_wrapper INFO     Inferring region to gene relationships
2022-11-08 14:33:43,373 R2G          INFO     Calculating region to gene importances, using GBM method
2022-11-08 15:15:38,230 R2G          INFO     Took 2514.856024980545 seconds
2022-11-08 15:15:38,234 R2G          INFO     Calculating region to gene correlation, using SR method
2022-11-08 15:55:22,372 R2G          INFO     Took 2384.1362385749817 seconds
2022-11-08 15:55:36,780 R2G          INFO     Done!
2022-11-08 15:55:37,256 SCENIC+_wrapper INFO     Inferring TF to gene relationships
2022-11-08 15:55:58,837 TF2G         INFO     Calculating TF to gene correlation, using GBM method
2022-11-08 21:12:56,955 TF2G         INFO     Took 19018.11377763748 seconds
2022-11-08 21:12:56,960 TF2G         INFO     Adding correlation coefficients to adjacencies.
2022-11-08 21:14:48,123 TF2G      

In [6]:
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    sys.stderr = open(os.path.join(scratch_dir, 'err2.log'), "w")
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = os.path.join(local_state_dir, 'TF_names_v_1.01.txt'),
        save_path = os.path.join(scratch_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = os.path.join(local_state_dir, 'bin'),
        n_cpu = 32,
        _temp_dir = os.path.join(scratch_dir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    dill.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)
    raise(e)

2022-11-08 23:39:53,733 SCENIC+_wrapper INFO     Created folder : /scratch/leuven/338/vsc33838/scenicplus
2022-11-08 23:39:53,734 SCENIC+_wrapper INFO     Merging cistromes
2022-11-08 23:43:34,522 SCENIC+_wrapper INFO     Getting search space
2022-11-08 23:43:37,172 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2022-11-08 23:43:57,658 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2022-11-08 23:43:59,183 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
2022-11-08 23:44:01,341 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
2022-11-08 23:44:18,365 R2G          INFO     Intersecting with regions.
2022-11-08 23:44:19,787 R2G          INFO     Calculating distances from region to gene
2022-11-08 23:

MemoryError: 

In [ ]:
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    sys.stderr = open(os.path.join(scratch_dir, 'err2.log'), "w")
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = os.path.join(local_state_dir, 'TF_names_v_1.01.txt'),
        save_path = os.path.join(scratch_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = os.path.join(local_state_dir, 'bin'),
        n_cpu = 46,
        _temp_dir = os.path.join(scratch_dir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    pickle.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj_f.pkl'), 'wb'), protocol=-1)
    raise(e)

2022-11-09 10:45:44,267 SCENIC+_wrapper INFO     Created folder : /scratch/leuven/338/vsc33838/scenicplus
2022-11-09 10:45:44,269 SCENIC+_wrapper INFO     Merging cistromes
2022-11-09 10:49:27,947 SCENIC+_wrapper INFO     Getting search space
2022-11-09 10:49:30,718 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2022-11-09 10:49:43,941 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2022-11-09 10:49:45,453 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
2022-11-09 10:49:47,582 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
2022-11-09 10:50:04,642 R2G          INFO     Intersecting with regions.
2022-11-09 10:50:06,081 R2G          INFO     Calculating distances from region to gene
2022-11-09 10:

In [ ]:
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    sys.stderr = open(os.path.join(scratch_dir, 'err3.log'), "w")
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = os.path.join(local_state_dir, 'TF_names_v_1.01.txt'),
        save_path = os.path.join(scratch_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = os.path.join(local_state_dir, 'bin'),
        n_cpu = 46,
        _temp_dir = os.path.join(scratch_dir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    pickle.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj_f.pkl'), 'wb'), protocol=-1)
    raise(e)

2022-11-09 19:51:07,600 SCENIC+_wrapper INFO     /scratch/leuven/338/vsc33838/scenicplus folder already exists.
2022-11-09 19:51:07,602 SCENIC+_wrapper INFO     Merging cistromes
2022-11-09 19:54:48,714 SCENIC+_wrapper INFO     Getting search space
2022-11-09 19:54:51,355 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2022-11-09 19:55:03,034 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2022-11-09 19:55:05,138 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
2022-11-09 19:55:07,256 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
2022-11-09 19:55:24,031 R2G          INFO     Intersecting with regions.
2022-11-09 19:55:25,475 R2G          INFO     Calculating distances from region to gene
2022-11-

In [ ]:
# LAST TRY
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    sys.stderr = open(os.path.join(scratch_dir, 'err6.log'), "w")
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = os.path.join(local_state_dir, 'TF_names_v_1.01.txt'),
        save_path = os.path.join(scratch_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = os.path.join(local_state_dir, 'bin'),
        n_cpu = 20,
        _temp_dir = os.path.join(scratch_dir, 'ray_spill'))
except Exception as e:
    #in case of failure, still save the object
    pickle.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj_f.pkl'), 'wb'), protocol=-1)
    raise(e)

2022-11-11 11:33:47,982 SCENIC+_wrapper INFO     /scratch/leuven/338/vsc33838/scenicplus folder already exists.
2022-11-11 11:33:47,984 SCENIC+_wrapper INFO     Merging cistromes
2022-11-11 11:37:25,335 SCENIC+_wrapper INFO     Getting search space
2022-11-11 11:37:28,765 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2022-11-11 11:37:42,849 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2022-11-11 11:37:44,376 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
2022-11-11 11:37:46,485 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
2022-11-11 11:38:02,635 R2G          INFO     Intersecting with regions.
2022-11-11 11:38:04,079 R2G          INFO     Calculating distances from region to gene
2022-11-

# Checkpoint

In [8]:
scplus_obj

SCENIC+ object with n_cells x n_genes = 11804 x 36066 and n_cells x n_regions = 11804 x 820223
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'n_cells', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
	metadata_cell:'celltype'
	menr:'CTX_topics_otsu_All', 'CTX_topics_otsu_No_promoters', 'DEM_topics_otsu_All', 'DEM_topics_otsu_No_promoters', 'CTX_topics_top_3_All', 'CTX_topics_top_3_No_promoters', 'DEM_topics_top_3_All', 'DEM_topics_top_3_No_promoters', 'CTX_DARs_All', 'CTX_DARs_No_promoters', 'DEM_DARs_All', 'DEM_DARs_No_promoters'

In [ ]:
pickle.dump(scplus_obj, open(os.path.join(scratch_dir, 'scenicplus/scplus_obj_f.pkl'), 'wb'), protocol=-1)

In [ ]:
import dill
scplus_obj = dill.load(open(os.path.join(scratch_dir, 'scenicplus/scplus_obj.pkl'), 'rb'))

# Downstream 

In [ ]:
from scenicplus.preprocessing.filtering import apply_std_filtering_to_eRegulons
apply_std_filtering_to_eRegulons(scplus_obj)
scplus_obj.uns['eRegulon_metadata_filtered'].head()

In [ ]:
from scenicplus.eregulon_enrichment import score_eRegulons
region_ranking = dill.load(open(os.path.join(work_dir, 'scenicplus/region_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
gene_ranking = dill.load(open(os.path.join(work_dir, 'scenicplus/gene_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
score_eRegulons(scplus_obj,
                ranking = region_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type= 'region',
                auc_threshold = 0.05,
                normalize = False,
                n_cpu = 5)
score_eRegulons(scplus_obj,
                gene_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type = 'gene',
                auc_threshold = 0.05,
                normalize= False,
                n_cpu = 5)

In [ ]:
from scenicplus.RSS import *
regulon_specificity_scores(
        scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_keys = ['Region_based'],
        selected_regulons = [x for x in scplus_obj.uns['selected_eRegulon']['Region_based'] if '-' not in x],
        out_key_suffix = '_filtered')

In [ ]:
plot_rss(scplus_obj, 'GEX_celltype_filtered', num_columns=2, top_n=10, figsize = (5, 10))